In [381]:
# Path ORAM 

from collections import defaultdict, Counter
import random

class Node(object): 
	def __init__(self, ary, idx, left, right): 
		self.val = ary
		self.idx = idx
		self.left = left 
		self.right = right
		
	def __repr__(self):
		return f"({self.idx},{self.val},{self.left},{self.right})"


# constructing a simple binary tree
# actually each of the int in the arrary should be a pointer to a block of size B
#    0    l0
#   /\
#  1  2   l1
#  /\ /\
# 3 4 5 6 l2

# n3 = Node([("", 0),("", 0),("", 0),("", 0)], 3, None, None) 
# n4 = Node([("", 0),("", 0),("", 0),("", 0)], 4, None, None)
# n5 = Node([("", 0),("", 0),("", 0),("", 0)], 5, None, None)
# n6 = Node([("", 0),("", 0),("", 0),("", 0)], 6, None, None)

# n1 = Node([("", 0),("", 0),("", 0),("", 0)], 1, n3, n4)
# n2 = Node([("", 0),("", 0),("", 0),("", 0)], 2, n5, n6)

# n0 = Node([("", 0),("", 0),("", 0),("", 0)], 0, n1, n2)

n3 = Node([], 3, None, None) 
n4 = Node([], 4, None, None)
n5 = Node([], 5, None, None)
n6 = Node([], 6, None, None)

n1 = Node([], 1, n3, n4)
n2 = Node([], 2, n5, n6)

n0 = Node([], 0, n1, n2)

print(n0)

(0,[],(1,[],(3,[],None,None),(4,[],None,None)),(2,[],(5,[],None,None),(6,[],None,None)))


In [382]:
def getHeight(root):
	'''
	Assumption: the given tree is a binary tree 
	Input: the root node of a tree 
	Output: the height of the tree 
	'''
	if not root: 
		return 0 

	leftHeight = getHeight(root.left)
	rightHeight = getHeight(root.right) 

	return max(leftHeight, rightHeight) + 1 

In [383]:
LEVELS = getHeight(n0)
print(LEVELS)

3


In [384]:
# Global set up 
N = 28 
Z = 4 # Z numbers of blocks within each bucket
global STASH 
STASH = []
random.seed(10)
# position = defaultdict(int, {k:random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1)) for k in range(N)})
position = defaultdict(int, {k:random.randrange(0, (pow(2, LEVELS) - 1)) for k in range(1,N+1)})

# The hard coded tree does not match the position map 
sorted_pos = defaultdict(list)
# {k : v if k in position else 0 for k,v in position.items()}

for key,val in position.items():
    sorted_pos[val].append(key)
    

In [385]:
print(position)
print(sorted_pos)

defaultdict(<class 'int'>, {1: 4, 2: 0, 3: 3, 4: 3, 5: 4, 6: 0, 7: 1, 8: 3, 9: 6, 10: 3, 11: 6, 12: 2, 13: 5, 14: 6, 15: 1, 16: 0, 17: 4, 18: 3, 19: 2, 20: 0, 21: 1, 22: 5, 23: 2, 24: 0, 25: 3, 26: 6, 27: 1, 28: 4})
defaultdict(<class 'list'>, {4: [1, 5, 17, 28], 0: [2, 6, 16, 20, 24], 3: [3, 4, 8, 10, 18, 25], 1: [7, 15, 21, 27], 6: [9, 11, 14, 26], 2: [12, 19, 23], 5: [13, 22]})


In [386]:
def concat_STASH(anotherL):
    for i in anotherL:
        STASH.append(i)
    return None 

In [387]:

def initialize_tree(root):
    if root is None:
        return None 

    # print("nodeval is {}".format(root.idx))
    data_ = [("", x) for x in sorted_pos[root.idx]]
    data = data_[:Z]
    concat_STASH(data_[Z:])
    print("STASH is {}".format(STASH))
    
    root.val = data 
    # print("node data is {}".format(root.val))
    initialize_tree(root.left)
    initialize_tree(root.right)
    return None

initialize_tree(n0)
n0

STASH is [('', 24)]
STASH is [('', 24)]
STASH is [('', 24), ('', 18), ('', 25)]
STASH is [('', 24), ('', 18), ('', 25)]
STASH is [('', 24), ('', 18), ('', 25)]
STASH is [('', 24), ('', 18), ('', 25)]
STASH is [('', 24), ('', 18), ('', 25)]


(0,[('', 2), ('', 6), ('', 16), ('', 20)],(1,[('', 7), ('', 15), ('', 21), ('', 27)],(3,[('', 3), ('', 4), ('', 8), ('', 10)],None,None),(4,[('', 1), ('', 5), ('', 17), ('', 28)],None,None)),(2,[('', 12), ('', 19), ('', 23)],(5,[('', 13), ('', 22)],None,None),(6,[('', 9), ('', 11), ('', 14), ('', 26)],None,None)))

In [388]:
def getPath(root, NodeId): 
	'''
	Input: root(root) node, and leaf node(NodeId)
	Output: all nodes along the path from leaf node to root node. 
	'''
	if not root:
		return []
	
	if root.idx == NodeId:
		return [(root.idx, root.val)] 

	leftPath = getPath(root.left, NodeId) 
	rightPath = getPath(root.right, NodeId)
 
	if leftPath:
		return leftPath +  [(root.idx, root.val)]
	if rightPath: 
		return rightPath + [(root.idx, root.val)]
	return []

In [389]:
print(getPath(n0, 6))

[(6, [('', 9), ('', 11), ('', 14), ('', 26)]), (2, [('', 12), ('', 19), ('', 23)]), (0, [('', 2), ('', 6), ('', 16), ('', 20)])]


In [390]:
# inspection 
def getDataInBlock(root, blockId):
  ''' 
  Get the data associated with a block ID in a given tree
  Input: root of a tree and a block ID
  Output: the data associated with the block ID 
  '''
  if root is not None:
    # it should be iterator here rather than fix length traversal 
    for elem in iter(root.val):
      if elem[1] == blockId:
        return elem[0]
    # for i in range(Z):
    #   if(root.val): #and len(root.val) == Z:
    #     if(root.val[i][1] == blockId):
    #       return root.val[i][0]

    lres = getDataInBlock(root.left, blockId)
    if lres is not None: 
      return lres 

    rres = getDataInBlock(root.right, blockId)
    if rres is not None:
      return rres

getDataInBlock(n0,16)

''

In [391]:
def getNodeAtLevel(root, leafidx, level):
  '''
  Input: the root of a tree, the leaf node index, a specific level
  Output: the node along the path from a given leaf node to root node at the given level 
  '''
  path = getPath(root, leafidx)
  path_len = len(path)
  if level > path_len - 1:
    return None
  return path[path_len-level-1]


In [392]:
def ReadBucket(root, leaf):
    '''
    read all of the (block, data) pair along the path from a leaf node to the root node 
    Input: the root of a tree, a leaf node, and a 

    '''
    # STASH = []
    for l in range(LEVELS):
        bucket = getNodeAtLevel(root, leaf, l)
        if bucket is not None:
            bucket = bucket[1]
            print("bucket is {}".format(bucket))
            for elem in bucket: 
                STASH.append(elem)
    return STASH
    
# ReadBucket(n0, 5)


In [393]:
def getCandidateBlocksHelper(root, leaf, blockID, level): 
	'''Output: (node.idx, node.val)'''
	print("--------------------------------------------------------------------------------------------inspection function")
	print("leaf is {} blockID is {} level is{}".format(leaf, blockID, level))
	print("current blockid {} maps to leaf {} at level {}".format(blockID,position[blockID], level))
	lhs = getNodeAtLevel(root, leaf, level)
	print("left hand side is ")
	print(lhs)
	print("right hand side is ")	
	rhs = getNodeAtLevel(root, position[blockID], level)
	print(rhs)
 
	if lhs is not None and rhs is not None:
		if lhs == rhs:
			return (getDataInBlock(root, blockID), blockID)
		else:
			return None 
	return None

In [394]:
def testing(leaf, blockid):
    for i in range(3):
        print("Testing at Level {}".format(i))
        print(getCandidateBlocksHelper(n0, leaf, blockid, i))        

In [395]:
def testall(): 
    for leaf,bucket in sorted_pos.items():
        print("---------------{}-------------".format(leaf))
        for bid in bucket:
            print("=========={}===========".format(bid))
            testing(leaf, bid)

# testall()


In [396]:
# {4: [1, 5, 17, 28], 0: [2, 6, 16, 20, 24], 3: [3, 4, 8, 10, 18, 25], 1: [7, 15, 21, 27], 6: [9, 11, 14, 26], 2: [12, 19, 23], 5: [13, 22]}
# 4 3 1 
# print(getCandidateBlocksHelper(n0, 3, 2, 0))
print(getCandidateBlocksHelper(n0, 3, 2, 1))
# print(getCandidateBlocksHelper(n0, 3, 4, 2))

--------------------------------------------------------------------------------------------inspection function
leaf is 3 blockID is 2 level is1
current blockid 2 maps to leaf 0 at level 1
left hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
right hand side is 
None
None


In [397]:
def getCandidateBlocks(root, leaf, level): 
    acc = []
    # print("from leaf {}".format(leaf))
    for idb in range(1, N + 1):
        print("idb is {}".format(idb))
        blocks = getCandidateBlocksHelper(root, leaf, idb, level)
        # print("blocks {} on the same path as leaf {} are blocks {}".format(idb, leaf, blocks))
        acc.append(blocks)
    res = [x for x in acc if x is not None]
    return res

In [398]:
blocks0 = getCandidateBlocks(n0, 3, 1)
# blocks1 = getCandidateBlocks(n0, 3, 2)
# blocks2 = getCandidateBlocks(n0, 4, 0)

blocks0
# blocks1


idb is 1
--------------------------------------------------------------------------------------------inspection function
leaf is 3 blockID is 1 level is1
current blockid 1 maps to leaf 4 at level 1
left hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
right hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
idb is 2
--------------------------------------------------------------------------------------------inspection function
leaf is 3 blockID is 2 level is1
current blockid 2 maps to leaf 0 at level 1
left hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
right hand side is 
None
idb is 3
--------------------------------------------------------------------------------------------inspection function
leaf is 3 blockID is 3 level is1
current blockid 3 maps to leaf 3 at level 1
left hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
right hand side is 
(1, [('', 7), ('', 15), ('', 21), ('', 27)])
idb is 4
--------------------------------------------------

[('', 1),
 ('', 3),
 ('', 4),
 ('', 5),
 ('', 7),
 ('', 8),
 ('', 10),
 ('', 15),
 ('', 17),
 (None, 18),
 ('', 21),
 (None, 25),
 ('', 27),
 ('', 28)]

In [399]:
def writeBackNodes(root, index, currlevel, tgtlevel, data):
	if root is None:
		return None

	if root.idx == index and currlevel == tgtlevel:
		retVal = root.val 
		root.val = data 
		return retVal

	nextlevel = currlevel + 1

	leftTree = writeBackNodes(root.left, index, nextlevel, tgtlevel, data)
	if leftTree: 
		return leftTree
  
	rightTree = writeBackNodes(root.right, index, nextlevel, tgtlevel, data)
	if rightTree:
		return rightTree

	return None  

In [400]:
# writeBackNodes(n0, 0, 2, 0, [('d', 0), ('a', 0), ('t', 0), ('a', 0)])
# print(n0)

In [401]:
def update_STASH(blockID, dataN, stsh):
    blockIndex = -1
    for (idx, pair) in enumerate(stsh):
        temp = list(pair)
        if temp[1] == blockID:
            blockIndex = idx 
    stsh[blockIndex] = (dataN, blockID)

In [402]:
def access(root, opCode, blockId, dataNew): 
	leafIdx = position[blockId]
	print("Node blockID {} maps to leaf {}".format(blockId, leafIdx))
	position[blockId] = random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1))
	print("new leaf is {}".format(position[blockId]) )
	
	STASH = ReadBucket(root, leafIdx)
	print("root is {}, leaf is {}".format(root, leafIdx))
	print("Current STASH is ")
	print(STASH)
 
	# read block a from STASH
	dataOld = getDataInBlock(root, blockId)
	print("old data is ")
	print(dataOld)
 
	if opCode == "wr": 
		# (check if the blockId is valid): do we assume a perfect binary tree? 
		# STASH = [item for item in STASH if item not in [STASH[blockId]]]
		updateNode = getNodeAtLevel(root, leafIdx, LEVELS - 1)
		update_STASH(blockId, dataNew, STASH)
		print("updated STASH is {}".format(STASH))
		print("========== node to be written is  ==========")
		print(updateNode)
 
	# write path 
	for l in reversed(range(LEVELS)):
		# insert additional blocks from the STASH to the path along the tree 
		
		candidateBlocks = getCandidateBlocks(root,  leafIdx, l)
		
		if (len(candidateBlocks) >= Z):
			writeBackSize = Z
		else:
			writeBackSize = len(candidateBlocks)

		writeBackBlocks = candidateBlocks[:writeBackSize]
		print("CandidateBlock at level {}".format(l))
		print(candidateBlocks)
		print("writeBacBlock at level {}".format(l))
		print(writeBackBlocks)
		updatedStash = candidateBlocks[writeBackSize:]
		# print("upated STASH is ")
		# print(updatedStash)

		writeBackNodes(root, leafIdx, 0, l, writeBackBlocks)
	return dataOld
	

In [403]:
def print_tree_status(status):
    print(status +" tree is ")
    print(n0)

In [404]:
print_tree_status("initialized")
oldD = access(n0, "wr", 16, "x")
print_tree_status("after first write")

initialized tree is 
(0,[('', 2), ('', 6), ('', 16), ('', 20)],(1,[('', 7), ('', 15), ('', 21), ('', 27)],(3,[('', 3), ('', 4), ('', 8), ('', 10)],None,None),(4,[('', 1), ('', 5), ('', 17), ('', 28)],None,None)),(2,[('', 12), ('', 19), ('', 23)],(5,[('', 13), ('', 22)],None,None),(6,[('', 9), ('', 11), ('', 14), ('', 26)],None,None)))
Node blockID 16 maps to leaf 0
new leaf is 5
bucket is [('', 2), ('', 6), ('', 16), ('', 20)]
root is (0,[('', 2), ('', 6), ('', 16), ('', 20)],(1,[('', 7), ('', 15), ('', 21), ('', 27)],(3,[('', 3), ('', 4), ('', 8), ('', 10)],None,None),(4,[('', 1), ('', 5), ('', 17), ('', 28)],None,None)),(2,[('', 12), ('', 19), ('', 23)],(5,[('', 13), ('', 22)],None,None),(6,[('', 9), ('', 11), ('', 14), ('', 26)],None,None))), leaf is 0
Current STASH is 
[('', 24), ('', 18), ('', 25), ('', 2), ('', 6), ('', 16), ('', 20)]
old data is 

updated STASH is [('', 24), ('', 18), ('', 25), ('', 2), ('', 6), ('x', 16), ('', 20)]
========== node to be written is  ==========
N